In [2]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import product
from typing import Dict, Tuple
from IPython.display import display
from matplotlib import pyplot as plt

from liquidity.features import compute_aggregate_features

from market_impact.util.utils import normalize_imbalances
from market_impact.response_functions import aggregate_impact
from market_impact.util.plotting.constants import EBAY_COLORS
from market_impact.finite_scaling.fss import mapout_scale_factors, transform
from market_impact.finite_scaling.fit import fit_scaling_form, fit_scaling_law
from market_impact.util.plotting.plot import plot_scaling_function, plot_collapsed_scaling_function

In [ ]:
# Package dependencies:
# https://github.com/anabugaenko/liquidity
# https://github.com/anabugaenko/market_impact

In [ ]:
# Automatically reload changes in package dependencies
%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')

<!--
Copyright 2022 Kenji Harada
-->
# Finitie-size scaling analysis
The statistics of price returns have been purported to exhibit robust scaling dynamics resembling those of systems in critical states  where the maximum increments of asset returns were previously shown to be limited only by a second upper cutoff due to, for example, the finite size of the system. We perform a finite-size scaling analysis  of the signed conditional aggregate impact of an order and characterize the critical exponents of relevant observables such as price returns signs and signed volume imbalances. In this notebook, we introduce a FSS based on the method of Leas-squares, see:

- Patzelt, F. and Bouchaud, J.P., 2018. 

## Finitie-size scaling of non-equilibrium phenomena


Upon introduction of properly scaled variables, the finite-size scaling (FSS) analysis extracts numerical values for critical exponents that describe a given critical phenomenon in a finitie-size system. More formally, the scaling assumption states that if a physical quantity is considered to obey scaling, it can be expressed as 
$$ 
y\left(x, T\right) = T^{\chi} \mathscr{F}\left(xT^{-\varkappa}\right), 
$$
where x is a variable describing a physical system of which size is T, and exponents $\chi$ and $\varkappa$ are critical exponents. $\mathscr{F}(\cdot)$ is a scaling function which too exhibits universality. Then the FSS analysis is an inference of critical parameters so that if we introduce new variables
$$ 
X \equiv xT^{-\kappa}, Y \equiv y /T^{\chi} =  yT^\chi 
$$
then the FSS law rewrites as 
$$
Y = \mathscr{F}(x)
$$
such that the data points collapse onto a single scaling function. 

## Infering the scaling function from data 
Because we may not know the form $\mathscr{F}$ $\textit{a prior}$ on grounds of theory, the statistical inference problem is acute and one has to assume both values of critical exponents as well as the form of the scaling function itself. Although $\mathscr{F}(𝑥)$ can in principal function of virtually any form, Patzelt and Bouchaud (2018) and Farmer, Gerig and Lillo (2008) find it is well appoximated by a sigmoidal:
\begin{align*}
\mathscr{F}(x) = \frac{x}{\left(1 + | x |^\alpha \right)^{\beta / \alpha}},
\end{align*}
which describes empirical observations for signed aggregate impact, where $\alpha$ and $\beta$ regulate the shape (steepness and symmetry) of $\mathscr{F}$.

## Empirical scaling of price returns
We are particualrly interested in the scaling laws governing the form of conditional aggregate price returns 
\begin{align*}
R\left(\Delta \mathcal{E}, T\right) \cong R_T \cdot \mathscr{F}\left(\frac{\Delta \mathcal{E}}{\mathcal{E}_T}\right).
\end{align*}
where and $R_T $ and $\mathcal{E}_T$ are unknown scaling factors that do not depend on 𝑥, but instead on the system size T. In fact, without imposing any assumptions, empiricism suggests a scaling law of the form
\begin{align*}
\mathcal{E}_T  \thicksim \mathcal{E}_DT^\varkappa, \\
    R_T \thicksim \mathcal{R}(1)T^\chi, 
\end{align*}
which yields the following scaling law
\begin{align*}
R \left(\Delta \mathcal{E}, T \right) = \mathcal{R}(1)T^\chi \cdot \mathscr{F}\left(\frac{\Delta \mathcal{E}}{\mathcal{E}_DT^\varkappa}\right),
\end{align*}
for aggregate price returns, where $\mathcal{R}(1)$ and $\mathcal{E}_D$ represent constants of unit dimension that define a characteristic length scale.

In [ ]:
# Constants 
DATA_RANGE = list(range(5, 151))
BINNING_FREQUENCIES = [5, 10, 20, 50, 100]

In [ ]:
# Load orderbook raw sample data
stocks = ['TSLA', 'AMZN', 'NFLX', 'MSFT', 'EBAY', 'AAPL', 'GOOG']

current_dir = os.path.abspath('.')
root_dir = os.path.join(current_dir, '..', '..')
data_dir = os.path.join(root_dir, 'data', 'market_orders')

stock_data = {}

# Loop through each stock
for stock in stocks:

    filename = f"{stock}-2017-NEW.csv"

    stock_file_path = os.path.join(data_dir, filename)

    # Read the CSV and store in the dictionary
    stock_data[stock] = pd.read_csv(stock_file_path)

# Access the dataframe using stock's ticker as key
tsla_raw_df = stock_data['TSLA']
amzn_raw_df = stock_data['AMZN']
nflx_raw_df = stock_data['NFLX']
msft_raw_df = stock_data['MSFT']
ebay_raw_df = stock_data['EBAY']
aapl_raw_df = stock_data['AAPL']
goog_raw_df = stock_data['GOOG']

In [ ]:
aapl_raw_df.head()

### Aggregate features 
We first coarse-grain the data into different binning frequencies T that represent different system sizes (in event time) by marginalize over microscopic degrees of freedom in the system to yield an effective coarse-grained description at long distances.

In [ ]:
# Compute aggregate features 
aggregate_features = compute_aggregate_features(aapl_raw_df.head(1000000), DATA_RANGE)
display(aggregate_features)

### Aggregate imapct
From aggegate features, we compute aggregate impact of market orders MO. In preprartion for FSS analysis., all impact data are automatically rescaled  rescaled each day by the corresponding values of $R(1)$ and the daily number $\mathcal{E}_D$.

In [ ]:
# Compute data for susceptibility
imbalance_column = "sign_imbalance"
aggregate_impact_data = aggregate_impact(aggregate_features, conditional_variable=imbalance_column)

display(aggregate_impact_data)

## Find shape parameters
Determine the shape parameters $\alpha$ and $\beta$ of scaling function $\mathscr{F}(\cdot)$ by fitting the</b>
scaling function for $\textit{all}$ T.

In [ ]:
# Prepare original data for fitting
t_values = aggregate_impact_data['T'].values
imbalance_values = aggregate_impact_data[imbalance_column].values
r_values = aggregate_impact_data['R'].values

# Fit data for all Ts
params = fit_scaling_form(t_values, imbalance_values, r_values)

In [ ]:
RT, VT, alpha, beta = params
print(f'RT: {RT}')
print(f'VT: {VT}')
print(f'alpha: {alpha}')
print(f'beta: {beta}')

In [ ]:
plot_scaling_function(
    aggregate_impact_data, 
    scaling_params=params,
    line_color=EBAY_COLORS.dark_color,
    markers_color="white", 
    plotting_func="scaling_form",
    imbalance_column=imbalance_column,
    binning_frequencies=BINNING_FREQUENCIES)

## Map-out scale factors
Once $\mathscr{F}(\cdot)$ is fixed, we can use the found $\alpha$ and $\beta$ to map out the scale factors as a function of T (i.e., for each system size T), which are well very approximated by power-laws of T.

In [ ]:
RT_series, VT_series, RT_fit_object, VT_fit_object = mapout_scale_factors(aggregate_impact_data, alpha=alpha, beta=beta, imbalance_column=imbalance_column)

In [ ]:
# Plot scale factors RN and QN
plt.scatter(RT_series['x_values'], RT_series['y_values'])
plt.scatter(VT_series['x_values'], VT_series['y_values'])
plt.loglog()


In [ ]:
RT_fit_object.powerlaw.fit_results()

In [ ]:
VT_fit_object.powerlaw.fit_results()

### Determine rescaling exponents
In order to determine the rescaling exponents $\chi$ and $\varkappa$, the shape of the scaling form is fitted for each T keeping the same/constant values of $\alpha$ and $\beta$,</b> which are well approximated by the power law.

In [ ]:
chi = RT_fit_object.powerlaw.params.alpha
kappa = VT_fit_object.powerlaw.params.alpha
print(chi)
print(kappa)

In [ ]:
RT_fit_object.powerlaw.plot_fit()

## Do FSS by method of Least-squares
We can now use the found empirical scaling law to perform the FSS.

In [ ]:
# Prepare original data for fitting
t_values = aggregate_impact_data['T'].values
imbalance_values = aggregate_impact_data[imbalance_column].values
r_values = aggregate_impact_data['R'].values

# Fit data for all Ts
params = fit_scaling_law(t_values, imbalance_values, r_values, reflect_y=False)

In [ ]:
chi, kappa, alpha, beta, CONST = params
print(f'chi: {chi}')
print(f'kappa: {kappa}')
print(f'alpha: {alpha}')
print(f'beta: {beta}')

We plot the scaling for different binning_frequencies. Aggregate impact after an order "appears" to grow linear in volume imbalance with increasing $T$.

In [ ]:
plot_scaling_function(
    aggregate_impact_data, 
    scaling_params=params,
    line_color=EBAY_COLORS.dark_color,
    markers_color="white", 
    imbalance_column=imbalance_column,
    binning_frequencies=BINNING_FREQUENCIES)

### Transform data
To do the fss by method of least-squares, we use optimized critical paramters to rescale the scaling function onto a single master curve by initially fitting the scaling law to all $T$.

In [ ]:
# Transform original data using found rescaling exponents chi 𝛘 and kapp ϰ
rescaled_data = transform(aggregate_impact_data, rescaling_params=params, imbalance_column=imbalance_column)

In [ ]:
plot_collapsed_scaling_function(
    rescaled_data,  
    scaling_params=params,
    line_color=EBAY_COLORS.dark_color, 
    markers_color="white", 
    imbalance_column=imbalance_column, 
    master_curve="Sigmoid",
    binning_frequencies=BINNING_FREQUENCIES)